#### Set up a  new conda environment 

In [ ]:
conda create -n pinterest_conda_env
conda activate pinterest_conda_env

Installed any libraries required to run the user_posting_emulation.py using 

In [ ]:
conda install library_name
conda install -c conda-forge library_name

#### Creating an EC2 instance 
Grabbed the key-pair from the created ec2 instance details section on AWS, and saved inside a .pem file.    
Followed instructions on AWS on how to connect to EC2 instance. Entering following code in CLI

In [ ]:
ssh -i "0a60b9a8a831-key-pair.pem" ec2-user@ec2-34-207-200-90.compute-1.amazonaws.com

#### Installing Kafka on the EC2 client 
First installed java and checked its version (java -version). Then downloaded and 'unzipped' kafka.

In [ ]:
sudo yum install java-1.8.0

In [ ]:
wget https://archive.apache.org/dist/kafka/2.8.1/kafka_2.12-2.8.1.tgz   # Grab the version of kafka to install
tar -xzf kafka_2.12-2.8.1.tgz                                           # To 'unzip' or 'untar' the file 
rm kafka_2.12-2.8.1.tgz                                                 # Remove the compressed file, keeping only uncompressed version

#### Installing IAM MSK authentication package on client EC2 machine.
Inside the Kafka installation folder and then in the libs folder.  
Downloaded the IAM MSK authentication package from Github, using the following command:

In [ ]:
cd kafka_2.12-2.8.1/
cd libs
wget https://github.com/aws/aws-msk-iam-auth/releases/download/v1.1.5/aws-msk-iam-auth-1.1.5-all.jar  # Download msk iam authentication file
export CLASSPATH=/home/ec2-user/kafka_2.12-2.8.1/libs/aws-msk-iam-auth-1.1.5-all.jar # Set class path environment variable for MSK authentication

Added above classpath environment variable to bashrc file to maintain across ec2 sessions 

In [ ]:
nano /home/ec2-user/.bashrc
source ~/.bashrc

![](/home/mhash/pinterest_data_pipeline_project/project_log/nano_bashrc_20231211.png)

In [ ]:
echo $CLASSPATH                          # To test, should return class path in configured correctly /home/ec2-user/kafka_2.12-2.8.1/libs/aws-msk-iam-auth-1.1.5-all.jar


#### To assume the \<UserId\>-ec2-access-role, which contains the necessary permissions to authenticate the MSK cluster
    
IAM console > Roles > \<UserId\>-ec2-access-role      
- copy ARN      

Trust relationships tab > Edit trust policy > Add a principal       
- Selected IAM roles as the Principal type , Replace ARN with the \<UserId\>-ec2-access-role ARN copied from ec2-access-role.


#### Configure Kafka client to use AWS IAM
modify client.properties file inside kafka_folder/bin directory

In [ ]:
cd kafka_2.12-2.8.1/
cd bin
nano client.properties

In [ ]:
# https://github.com/aws/aws-msk-iam-auth instructions on configuring a Kafka client to use AWS IAM with AWS_MSK_IAM mechanism

# Sets up TLS (Transport Layer Security) for encryption (cryptographic protocol) and SASL (Simple Authentication and Security Layer) for authN (framework for authentication and data security in Internet protocols).
security.protocol = SASL_SSL

# Identifies the SASL mechanism to use.
sasl.mechanism = AWS_MSK_IAM

# Binds SASL client implementation.
sasl.jaas.config = software.amazon.msk.auth.iam.IAMLoginModule required awsRoleArn="arn:aws:iam::584739742957:role/0a60b9a8a831-ec2-access-role";

# Encapsulates constructing a SigV4 signature based on extracted credentials.
# The SASL client bound by "sasl.jaas.config" invokes this class.
sasl.client.callback.handler.class = software.amazon.msk.auth.iam.IAMClientCallbackHandler`

#### Creating Kafka Topics

Using the MSK Management Console to get cluster information
Amazon MSK > pinterest-msk-cluster > View client information and save:  
- Bootstrap servers Private endpoint (single-VPC)
- Plaintext Apache Zookeeper connection string  

topic names: 
- 0a60b9a8a831.pin for the Pinterest posts data
- 0a60b9a8a831.geo for the post geolocation data
- 0a60b9a8a831.user for the post user data


In [ ]:
./kafka-topics.sh --bootstrap-server b-1.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098 --command-config client.properties --create --topic 0a60b9a8a831.pin
./kafka-topics.sh --bootstrap-server b-2.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098 --command-config client.properties --create --topic 0a60b9a8a831.geo
./kafka-topics.sh --bootstrap-server b-3.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098 --command-config client.properties --create --topic 0a60b9a8a831.user

#### Connecting the MSK cluster to an S3 bucket.    
The s3 bucket our data will be saved in is user-0a60b9a8a831-bucket (IAM role already set up to write to s3 bucket )

first Download confluent.io 

In [ ]:
# assume admin user privileges
sudo -u ec2-user -i 

# create directory where we will save our connector 
mkdir kafka-connect-s3 && cd kafka-connect-s3 

# download connector from Confluent
wget https://d1i4a15mxbxib1.cloudfront.net/api/plugins/confluentinc/kafka-connect-s3/versions/10.0.3/confluentinc-kafka-connect-s3-10.0.3.zip 

# copy connector to S3 bucket 
aws s3 cp ./confluentinc-kafka-connect-s3-10.0.3.zip s3://user-0a60b9a8a831-bucket/kafka-connect-s3/

#### Created custom plugin in the MSK Connect console   
MSK console > MSK Connect section  > Custom plugins > Create custom plugin.
Choose bucket where Confluent connector ZIP file is (s3://user-0a60b9a8a831-bucket/kafka-connect-s3/confluentinc-kafka-connect-s3-10.0.3.zip). Name plugin 0a60b9a8a831-plugin.

#### Creating a connector in MSK connect using custom plugin    
 
MSK connect > Customised plugins > choose 0a60b9a8a831-plugin > Create connector >
created a connector with the following 
Connector name : 0a60b9a8a831-connector
Description – optional : N/A

Specifically your bucket name should be user-<your_UserId>-bucket
topics.regex field in the connector configuration. Make sure it has the following structure: <your_UserId>.*. 
choose the IAM role used for authentication to the MSK cluster in the Access permissions tab. Remember the role has the following format <your_UserId>-ec2-access-role.
